Я решила найти все предметы искусства, связанные с богиней Венерой (Афродитой), и различными амазонками (так как их достаточно много, многие из них также богини), находящиеся в музеях топ-5 городов-миллиоников России.

Первым делом импортируем необходимые библиотеки.

In [1]:
!pip install flatten_json
import requests
import pandas as pd
import json
from flatten_json import flatten
import time

Объявляем переменные для наборов данных и авторизации через ключ API. Далее - задаем список переменных, которые в дальнейшем передадим в цикл.

In [2]:
list_of_all_dfs = []
key = "6ddd0fda7f9298dcfa843f082b2972fd6875ddea9db9b9b5ff2572439193ecc4"

In [3]:
keywords = ["венера", "афродита", "амазонка"]

cities = ["москва", "санкт-петербург", "новосибирск", "екатеринбург", "казань"]

Далее мы создадим функцию для выполнения запроса. Так как на первых стадиях возникли проблемы с добавлением нескольких параметров, я приняла решение работать с собственной функции для удобства. К тому же, это облегчит читаемость кода, так как отсутствуют повторы для каждого параметра.

В самой функции также создаю словарь.

In [4]:
def make_request(keyword, city, s):

    params = {
        "data.name": {"$search": keyword},
        "data.museum.name": {"$search": city},
        "data.productionPlace": {"$ne": "россия"}
    }
    
    encoded_params = json.dumps(params, ensure_ascii=False) # Преобразут словарь с параметрами в JSON
    url = f"https://opendata.mkrf.ru/v2/museum-exhibits/$?f={encoded_params}&s={s}&l=1000"
    response = requests.get(url, headers={"X-API-KEY": key}, timeout=15)
    response.raise_for_status()
    return response.json()

Далее работает тройной вложенный цикл: ключевые слова -> города -> страницы, чтобы пройтись по всем параметрам.
Использую библиотеку time и функцию sleep для паузы между реквестами, чтобы избежать ошибок (в частности, я столкнулась с ошибкой 429).

In [5]:
for keyword in keywords:
    for city in cities:
        print(f"Ищем: '{keyword}' в городе {city}")
        
        total_records = 0
        for s in range(0, 222000, 1000):
            time.sleep(0.3)
            
            data = make_request(keyword, city, s)

            if data['data']:
                records = [flatten(record) for record in data['data']]
                df = pd.DataFrame(records)
                
                list_of_all_dfs.append(df)
                total_records += len(df)
                
                print(f"Найдено {len(df)} предметов")
            else:
                break
                
print("Done")

Ищем: 'венера' в городе москва
Найдено 625 предметов
Ищем: 'венера' в городе санкт-петербург
Найдено 624 предметов
Ищем: 'венера' в городе новосибирск
Найдено 52 предметов
Ищем: 'венера' в городе екатеринбург
Найдено 15 предметов
Ищем: 'венера' в городе казань
Найдено 16 предметов
Ищем: 'афродита' в городе москва
Найдено 15 предметов
Ищем: 'афродита' в городе санкт-петербург
Найдено 91 предметов
Ищем: 'афродита' в городе новосибирск
Ищем: 'афродита' в городе екатеринбург
Найдено 2 предметов
Ищем: 'афродита' в городе казань
Ищем: 'амазонка' в городе москва
Найдено 27 предметов
Ищем: 'амазонка' в городе санкт-петербург
Найдено 84 предметов
Ищем: 'амазонка' в городе новосибирск
Найдено 2 предметов
Ищем: 'амазонка' в городе екатеринбург
Найдено 1 предметов
Ищем: 'амазонка' в городе казань
Done


In [6]:
list_of_all_dfs

[                          _id  nativeId                      hash   data_id  \
 0    5c3e1a4f93fa687ca4b1fedb  10298202  2017-12-29T09:16:28.522Z  10298202   
 1    5c3e1a7e93fa687ca4b47fc9  10371479  2018-01-15T07:32:33.256Z  10371479   
 2    5c3e1a8a93fa687ca4b4de5a  10375887  2018-01-15T09:29:33.125Z  10375887   
 3    5c3e1a8d93fa687ca4b52d72  10386926  2018-01-16T06:52:10.985Z  10386926   
 4    5c3e1ada93fa687ca4b8ed04  10471600  2018-01-23T07:28:09.912Z  10471600   
 ..                        ...       ...                       ...       ...   
 620  5c3e17bc93fa687ca48c21e9   9279287  2017-10-16T12:48:45.676Z   9279287   
 621  5c3e17bc93fa687ca48c2215   9279295  2017-10-16T12:48:45.676Z   9279295   
 622  5c3e17bc93fa687ca48c2256   9279306  2017-10-16T12:48:45.676Z   9279306   
 623  5c3e17bc93fa687ca48c2282   9279313  2017-10-16T12:48:45.676Z   9279313   
 624  5c3e195993fa687ca4a36c75   9953949  2017-12-07T15:07:39.462Z   9953949   
 
                                      

In [7]:
df_all = pd.DataFrame()
for df in list_of_all_dfs:
    df_all = pd.concat([df_all,df])

df_all

,_id,nativeId,hash,data_id,data_name,data_description,data_partsCount,data_regNumber,data_invNumber,data_gikNumber,...,changes_0_diff_0_value_images_5_url,changes_0_diff_0_value_images_6_url,changes_0_diff_0_value_images_7_url,changes_0_diff_0_value_images_8_url,changes_0_diff_34_op,changes_0_diff_34_path,changes_0_diff_35_op,changes_0_diff_35_path,data_length,changes_0_diff_4_value_id
0,5c3e1a4f93fa687ca4b1fedb,10298202,2017-12-29T09:16:28.522Z,10298202,Лист с тремя изображениями: «Аякс уносит тело ...,изображение в верхней левой части листа- пятер...,1,10191707,Э-17343,ГМП КП-26444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5c3e1a7e93fa687ca4b47fc9,10371479,2018-01-15T07:32:33.256Z,10371479,"Скульптура ""Венера""",фигура стоящей в полный рост обнаженной девушк...,1,10265363,ФР 12937,ГМК КП 4722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5c3e1a8a93fa687ca4b4de5a,10375887,2018-01-15T09:29:33.125Z,10375887,"Скульптура ""Венера Италийская""","Венера, выходящая из купальни и прижимающую к ...",1,10269614,О-3480,МГОМЗ КП-18639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5c3e1a8d93fa687ca4b52d72,10386926,2018-01-16T06:52:10.985Z,10386926,"Макет. Спускаемый аппарат АМС ""Венера-4"". М 1:1.",NaN,1,10280612,ММК-VI-М 111,ММК ГИК 473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5c3e1ada93fa687ca4b8ed04,10471600,2018-01-23T07:28:09.912Z,10471600,"Скульптура ""Венера""",NaN,1,10362750,ФР 12356,ГМК КП 2307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,5c3e1d2d93fa687ca4dbdc32,7867083,2017-05-19T11:46:55.331Z,7867083,Открытка. Амазонка. Национальный музей. Варшав...,NaN,1,7758954,I-Б-44118/-р,ГМИ СПб 443825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,5c3e15ee93fa687ca4725d65,7919281,2017-05-29T11:18:24.268Z,7919281,"Эскиз костюма Амазонки к балету ""Царь Кандавл""...","Женская фигура стоя в рост, 3/4 влево. Голова ...",1,7805883,ОР 3431,СПбГМТМИ ГИК 7084/69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,637d3fe4f5b1ca408c58990e,39684145,2022-11-16T10:09:12.503Z,39684145,Лежащая амазонка.,"скульптура изображает обнаженную женщину, лежа...",1,39463268,,НГКМ ОФ-23932/48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,637d3fe4f5b1ca408c589914,39684159,2022-11-16T10:09:12.503Z,39684159,Лежащая амазонка.,скульптура представляет собой эскиз фигуры леж...,1,39463262,,НГКМ ОФ-23932/42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Также создам CSV файл, в котором будет храниться мой датасет, чтобы загрузить в гитхаб.

In [8]:
final_df.to_csv('museums_search.csv', index=False)

NameError: name 'final_df' is not defined

Далее проведу анализ.
Перед ним мне надо посмотреть, насколько полученные данные удобны для анализа, а также учесть полезность всех столбцов (их более 400).

In [ ]:
df_all.columns

Вижу, что столбцов, несущих важную информацию, которую можно анализировать, не так много. Помимо этого, очень много столбцов с пропусками. 
Чтобы провести анализ данных важных столбцов, создам список, в который добавлю столбцы датасета, с которыми можно работать.

In [ ]:
columns = [
    '_id', 'data_name', 'data_description', 'data_regNumber', 
    'data_museum_name', 'data_authors_0', 'data_startDate', 
    'data_technologies_0', 'data_typology_name', 'data_images_0_url'
]

ds = df_all[columns]
ds

Разобьем столбец data_startDate и достанем из него год для анализа. Так же сделаем и с веком.

In [ ]:
ds['year_start'] = pd.to_datetime(ds['data_startDate'], errors='coerce').dt.year.astype('Int64').fillna(0)
ds['century'] = (ds['year_start'] // 100) + 1
ds

In [ ]:
ds['century'].value_counts().sort_values(ascending=False)

#### Разведочный анализ

In [ ]:
print(len(ds)) # Всего записей в датасете
print(ds['data_museum_name'].nunique()) # Уникальных музеев в датасете
print(ds.dtypes) # Типы данных в датасете

#### Группировка

In [ ]:
# Количество экспонатов в каждом музее

ds["data_museum_name"].value_counts().reset_index().head(10)

In [ ]:
# Распространие типологий

ds["data_typology_name"].value_counts().reset_index().head(10)

In [ ]:
# Музеи по разнообразию технологий
ds.groupby('data_museum_name')['data_technologies_0'].nunique().sort_values(ascending=False).reset_index().head(10)

In [ ]:
# Самые популярные технологии в 20 веке
ds[ds['century'] == 20]['data_technologies_0'].value_counts().head(5).reset_index()

In [ ]:
# Средний год создания для каждого автора
ds.groupby('data_authors_0')['year_start'].mean().sort_values(ascending=False).reset_index()

Также сделаем группировку, с помощью которой посчитаем среднее число и медиану.

In [ ]:
# Средний и медианный год создания по технологиям
ds.groupby('data_technologies_0')['year_start'].agg(['mean', 'median', 'count']).sort_values('count', ascending=False)

In [ ]:
# Среднее и медиана по музеям и векам
ds.groupby(['data_museum_name', 'century'])['year_start'].agg(['mean', 'median', 'size']).reset_index()

Далее построю графики для визуализации некоторых моментов.

In [ ]:
# Экспортирую необходимые библиотеки
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
# Топ-10 музеев по количеству экспонатов
museum_counts = ds["data_museum_name"].value_counts().reset_index().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x='count', y='data_museum_name', data=museum_counts, palette='flare')
plt.title('Топ-10 музеев по количеству экспонатов')
plt.xlabel('Количество экспонатов')
plt.ylabel('Музей')
plt.show()

In [ ]:
# Распределение типологий экспонатов
typology = ds["data_typology_name"].value_counts().reset_index().head(10)

plt.figure(figsize=(10, 8))
plt.pie(typology['count'], labels=typology['data_typology_name'], 
        colors=sns.color_palette('pastel'),
        wedgeprops={'linewidth': 1, 'edgecolor': 'white'})
plt.title('Распределение типологий экспонатов (топ-10)')
plt.show()

In [ ]:
# Топ-10 музеев по разнообразию технологий
museum_tech = ds.groupby('data_museum_name')['data_technologies_0'].nunique().sort_values(ascending=False).reset_index().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x='data_technologies_0', y='data_museum_name', data=museum_tech, palette='mako')
plt.title('Топ-10 музеев по разнообразию технологий')
plt.xlabel('Количество уникальных технологий')
plt.ylabel('Музей')
plt.show()

In [ ]:
# Популярные технологии XX века, топ-5
tech_20th = ds[ds['century'] == 20]['data_technologies_0'].value_counts().head(5).reset_index()

plt.figure(figsize=(10, 4))
sns.barplot(x='count', y='data_technologies_0', data=tech_20th, palette='rocket')
plt.title('Топ-5 технологий XX века')
plt.xlabel('Количество экспонатов')
plt.ylabel('Технология')
plt.show()

In [ ]:
# Топ-10 технологий по годам
tech_stats = ds.groupby('data_technologies_0')['year_start'].agg(['mean', 'median', 'count']).sort_values('count', ascending=False).head(10)

plt.figure(figsize=(10, 6))
tech_stats[['mean', 'median']].plot(kind='barh')
plt.title('Топ-10 технологий: средний и медианный год')
plt.xlabel('Год')
plt.ylabel('Технология')
plt.legend(['Средний год', 'Медианный год'])
plt.show()

### Выводы

Москва и Санкт-Петербург доминируют в рейтингах по количеству экспонатов и разнообразию технологий. Это подтверждает их статус ключевых культурных центров России, именно в этих городах и находятся самые важные предметы искусства.
Наиболее распространены предметы нумизматики, графика и печатная продукция. Это подчеркивает, что исторические и документальные коллекции, связанные с богинями и амазонками, более распространены, нежели живопись и скульптура - вероятно, из-за сложности хранения и реставрации таких объектов.
Среди технологий 20 века лидируют бумага, алюминий и офсетная печать, что связано с промышленным развитием и массовым производством - то есть предметы коллекции, каким-либо образом связанные с Венерой / Афрродитой / амазонками, в 20 году были сделаны именно из этих технологий.